# Данные

In [2]:
import csv
from collections import defaultdict
import random
import json
random.seed(42)

In [3]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [4]:
import scipy as sp
import scipy.sparse
import scipy.sparse.linalg

In [5]:
#Словари для основной выборки
user_to_items = defaultdict(set)
item_to_users = defaultdict(set)

#Словари для тестовой выборки
test_user_to_items = defaultdict(set)
test_item_to_users = defaultdict(set)

with open("data/train_likes.csv") as datafile:
    for like in csv.DictReader(datafile):
        # Кидаем монетку. В зависимости от результата кладём в обучение или тест
        if random.random() < 0.90:
            user_to_items[like['user_id']].add(like['item_id'])
            item_to_users[like['item_id']].add(like['user_id'])
        else:
            test_user_to_items[like['user_id']].add(like['item_id'])
            test_item_to_users[like['item_id']].add(like['user_id'])

In [6]:
all_items = set(item_to_users.keys()) | set(test_item_to_users.keys())
all_users = set(user_to_items.keys()) | set(test_user_to_items.keys())

In [7]:
user_to_i = {user: i for i, user in enumerate(all_users)}
item_to_i = {item: i for i, item in enumerate(all_items)}
all_users_list = list(all_users)
all_items_list = list(all_items)

In [8]:
matrix = sp.sparse.lil_matrix((len(all_users), len(all_items)))
test_matrix = sp.sparse.lil_matrix((len(all_users), len(all_items)))
for user, items in user_to_items.items():
    for item in items:
        matrix[user_to_i[user], item_to_i[item]] = True
for user, items in test_user_to_items.items():
    for item in items:
        test_matrix[user_to_i[user], item_to_i[item]] = True
matrix = matrix.tocsr()
test_matrix = test_matrix.tocsr()

In [9]:
matrix

<55863x23891 sparse matrix of type '<class 'numpy.float64'>'
	with 99582 stored elements in Compressed Sparse Row format>

In [10]:
u, s, vt = sp.sparse.linalg.svds(matrix.astype(np.float32), k=100)

In [11]:
u.shape, s.shape, vt.shape

((55863, 100), (100,), (100, 23891))

In [12]:
films = json.load(open('data/items.json'))
films = {a['id']:a for a in films}
for a in films.values():
    del a['id']

In [13]:
features = set()
for f in films.values():
    features|=set(f.keys())

In [14]:
features|=set(range(15))

In [15]:
feature_to_i = {feature: i for i, feature in enumerate(features)}

# Фильтрация пользователей
* Значительная часть пользователей имеет всего 1-2 просмотра. При всём желании, рекоммендовать им что-либо осмысленное при помощи рассматриваемого здесь метода мы вряд ли сможем. Для простоты вычислений, удалим их из выборки.
* Важно понимать, что качество на оставшихся пользователях скорее всего будет выше, чем на первоначальной выборке.

In [16]:
min_items_per_user = 2
from copy import copy
for user in copy(test_user_to_items).keys():
    
    n_items_per_user = len(user_to_items[user]) + len(test_user_to_items[user])
    
    if n_items_per_user <= min_items_per_user:
        del user_to_items[user]
        del test_user_to_items[user]

### Рекоммендующая функция
Позволим себе немного вольности: наша функция будет возвращать не вероятности, а список фильмов в порядке убывания "рекомендованности".

* Рекоммендованность фильма item пользователю user посчитаем так:
  * Для каждого фильма, полайканного пользователем user, найдём других людей, которым понравился фильм.
  * Сложим всех таких "друзей по лайкам" вместе и назовём соседями (__neighborhood__) пользователя.
  * Для фильма item узнаем его аудиторию - множество пользователей, которые его лайкнули
  * Пригодность фильма пользователю - то, насколько "друзьям по лайкам" пользователя нравится этот фильм.

Для примера, будем использовать косинусную меру расстояния
  
$ cos(u_{film}, u_{neighborhood}) = $ =$ u_{film} \cdot u_{neighborhood} \over |u_{film}| |u_{neighborhood}| $


$u_{neighborhood}$ зависит только от пользователя, но не от фильма, поэтому при сравнении фильмов по пригодности для одного пользователя, его можно исключить из формулы для простоты вычислений.

$ similarity(u_{film}, u_{neighborhood}) = $ $  u_{film} \cdot u_{neighborhood} \over |u_{film}| $
  
  
Распишем формулу подробно:

$ similarity(u_{film}, u_{neighborhood}) = $ $ \sum _{u_i} [u_i \in u_{film}] \cdot [u_i \in u_{neighborhood}] \over |u_{film}|  $

* u_i - очередной пользователь (в цикле по всем пользователям)
  
Выражение $[u_i \in u_{neighborhood}]$ здесь означает "сколько раз очередной пользователь входит в множество друзей по лайкам"
  
  

In [17]:
from math import sqrt
from collections import Counter

def recommend(user, n_best = 10):
    user_items = user_to_items[user]
    
    neighborhood = Counter()
    for item in user_items:
        neighborhood.update(item_to_users[item])
    
    #словарь {фильм -> пригодность фильма пользователю}
    item_similarities = {}
    
    for item in all_items:
        if item in user_items: continue
        item_users = item_to_users[item]
        if len(item_users) == 0: continue
        
        n_common_users = sum(neighborhood[user] for user in item_users)
        similarity = float(n_common_users) / sqrt(len(item_users))
        item_similarities[item] = similarity
    
    items_sorted = sorted(all_items, key = lambda x: item_similarities.get(x, 0),reverse = True)
    
    return items_sorted[:n_best]

In [18]:
user_to_int = dict()
for i, u in enumerate(all_users):
    user_to_int[u] = i 

In [19]:
#Making dataset
user_features = defaultdict(lambda:defaultdict(lambda:0))
for u in list(user_to_items.keys())[0:]:
    for item in user_to_items[u]:
        if item in films:
            for feature, value in films[item].items():
                if feature != 'id':
                    if feature=='genre':
                        user_features[u][value] += 1
                    else:    
                        user_features[u][feature]+=value
    for f in user_features[u]:
        user_features[u][f]/=len(user_to_items)

In [20]:
def generate_samples(X_size = 100):
    X = sp.sparse.lil_matrix((X_size, 2*len(feature_to_i)+2))
    Y = np.zeros(X_size, dtype='int8')

    for i in range(X_size):
        film =''
        while film not in films: 
            result = random.randint(0, 1)
            user = random.choice(all_users_list)
            if result==0:
                film = random.choice(all_items_list)
                if film in user_to_items[user]:
                    result = 1
            else:

                film = random.choice(list(user_to_items[user]|{''}))


        for f,val in user_features[user].items():
            X[i, 2*feature_to_i[f]] = int(val)
        for f,val in films[film].items():
            X[i, 2*feature_to_i[f]+1] = int(val)
        Y[i] = result
    return X, Y

In [21]:
rf = RandomForestClassifier(n_estimators=100, n_jobs=2)

In [22]:
for i in range(1):
    X, Y = generate_samples(100)
    rf.fit(X, Y)
    print(i)

0


In [23]:
X, Y = generate_samples(10)
rf.predict_proba(X), Y

(array([[ 0.81,  0.19],
        [ 0.73,  0.27],
        [ 0.78,  0.22],
        [ 0.82,  0.18],
        [ 0.62,  0.38],
        [ 0.24,  0.76],
        [ 0.77,  0.23],
        [ 0.66,  0.34],
        [ 0.83,  0.17],
        [ 0.79,  0.21]]), array([0, 0, 1, 1, 0, 1, 1, 0, 0, 1], dtype=int8))

In [24]:
def user_film_mk3(user, item, debug=False):
    X_size = 1
    X = sp.sparse.csc_matrix((X_size, 2*len(feature_to_i)+2))
    for f,val in user_features[user].items():
        X[i, 2*feature_to_i[f]] = int(val)
    if item not in films:
        return 0 
    for f,val in films[item].items():
        X[i, 2*feature_to_i[f]+1] = int(val)
    #probs = rf.predict_proba(X)
    if debug:
        print(probs)
    #return probs[0][0]
    return 0

In [25]:
def user_film_mk1(user, item, debug=False):
    #print(len(user_features[user]))
    cursum = 0
    curcnt = 0
    if item in films:
        for feature, value in films[item].items():
            if type(value) is int:
                curcnt+=1
                if feature=='genre':
                    #print(user_features[user])
                    cursum+=user_features[user][value]
                else:    
                    cursum+=value*user_features[user][feature]
    if curcnt==0:
        curcnt+=1
    #cursum/=curcnt
    if debug:
        print(cursum)
    return cursum

In [26]:
def user_film_mk2(user, item, debug=False):
    ui = user_to_i[user]
    ii = item_to_i[item]
    if debug:
        print(ui, ii)
    a = np.dot(u[ui,:] * s, vt[:,ii])
    return a

In [27]:
user_film_mk3('17db499bbbd058da91bddcaea365be5c', '98bdfa4bcad95291e88d56521b620acc', debug=True)

/opt/anaconda/lib/python3.5/site-packages/scipy/sparse/compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


NameError: name 'probs' is not defined

In [ ]:
def generic_recommend(user_flim_function):
    def recommend(user, n_best = 10, debug=False):
        user_items = user_to_items[user]

        item_similarities = {}
        for item in all_items:
            if item in user_items: continue
            item_users = item_to_users[item]
            if len(item_users) == 0: continue

            item_similarities[item] = user_flim_function(user, item)

        items_sorted = sorted(all_items, key = lambda x: item_similarities.get(x, 0),reverse = True)
        if debug:
            for a in  items_sorted[:n_best]:
                print((a, item_similarities.get(a, 0)))
        return items_sorted[:n_best]
    return recommend

In [ ]:
generic_recommend(user_film_mk2)('1a337111f63f6a7f86cebf6b2ad3d732', debug=True)
user_film_mk2('1a337111f63f6a7f86cebf6b2ad3d732', 'c745ed11b94ed93f3008897c75240de3', debug=True)

In [ ]:
generic_recommend(user_film_mk1)('1a337111f63f6a7f86cebf6b2ad3d732', debug=True)
user_film_mk1('1a337111f63f6a7f86cebf6b2ad3d732', 'c745ed11b94ed93f3008897c75240de3', debug=True)

In [ ]:
generic_recommend(user_film_mk3)('1a337111f63f6a7f86cebf6b2ad3d732', debug=True)
user_film_mk3('1a337111f63f6a7f86cebf6b2ad3d732', 'c745ed11b94ed93f3008897c75240de3', debug=True)

In [ ]:
list(test_user_to_items.items())[1]

In [ ]:
def recommend_dummy(user, n_best = 10):
    item_similarities = []
    for item in all_items:
        #пропустим те фильмы, которые пользователь уже просмотрел, если нас об этом попросили
        if item in user_to_items[user]: continue
        item_similarities.append(item)
         
    random.shuffle(item_similarities)
    #вернём n_best наиболее пригодных
    #print(items_sorted[:n_best])
    return item_similarities[:n_best]

# Оценка качества - map@k

In [45]:
check_quality(recommend_dummy,10,200)

0 / 200
100 / 200
AP@10 = 0.0


In [54]:
check_quality(generic_recommend(user_film_mk1),5,200)

0 / 200
100 / 200
AP@5 = 0.005583333333333333


In [55]:
check_quality(generic_recommend(user_film_mk2),5,200)

0 / 200
100 / 200
AP@5 = 0.026083333333333333


In [56]:
check_quality(recommend,5,200)

0 / 200
100 / 200
AP@5 = 0.019266666666666665


In [49]:
len(test_user_to_items)

2732

In [43]:
def APatK (actual_likes,recommendation_list, K =10):
    """Посчитать Average Precision at K"""

    countRelevants = 0
    sum_of_precisions = 0.0
    
    for i in range(min(K,len(recommendation_list))):
        currentk = i + 1.0
        if recommendation_list[i] in actual_likes:
            countRelevants+=1
        precisionAtK = countRelevants / currentk 
        sum_of_precisions += precisionAtK
        
    return sum_of_precisions / K 

In [44]:
#сколько рекоммендаций рассматриваем
def check_quality(function, K = 10, max_n_users = 500):
    APatK_per_user = []
    user_list = list(test_user_to_items.keys())[:max_n_users]
    for i, user in enumerate(user_list):
        #фильмы, которые пользователю на самом деле нравятся
        test_items = test_user_to_items[user]

        #Выдать топ-K рекоммендаций
        recommendation_list = function(user,n_best=K)

        #Посчитать ap@k
        user_APatK = APatK(test_items, recommendation_list,K=K)

        #и сложить в коробку
        APatK_per_user.append(user_APatK)

        #Progress bar
        if i % 100 ==0:
            print(i,'/',max_n_users)

        if i > max_n_users:
            break

    print('AP@{} = {}'.format(K, sum(APatK_per_user)/len(APatK_per_user)))


# Notes
* Кроме качества рекоммендаций, map@k ещё зависит от доли тестовой выборки, фильтрации и от самого K. Сравнивать качество разных алгоритмов имеет смысл только при одинаковом K и тестовой выборке.
* Давать полезные рекоммендации пользователям с малым числом просмотров тоже можно: например, можно выдавать наиболее популярные в целом фильмы.
* Разделение на обучение/тест честнее делать на по времени: первые 70% (например) лайков в обучение, остальные в тест. Это ближе к реальной жизни, когда вы сначала обучаете модель на логах, а потом применяете на новых сессиях пользователей.